In [10]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from numpy import pi

In [11]:
def get_qft(qreg):
    n = len(qreg)
    qft = QuantumCircuit(qreg)
    i = n - 1
    while i >= 0:
        qft.h(qreg[i])
        j=i-1
        while j >= 0:
            qft.cp(pi/2**(i-j),qreg[j],qreg[i])
            j-=1
        i-=1
    for k in range(n//2):
        qft.swap(qreg[k],qreg[n-k-1])
    return qft

In [12]:
def iqft_gater(qreg):
    iqft_circuit = get_qft(qreg).inverse()
    iqft_gate = iqft_circuit.to_gate(label="IQFT")
    return iqft_gate

In [13]:
qft = get_qft(QuantumRegister(4,'q'))
qft.draw()

┌───┐»
q_0: ─────────────────────────────■─────────────────■─────────────■───────┤ H ├»
                                  │                 │       ┌───┐ │P(π/2) └───┘»
q_1: ───────────────■─────────────┼────────■────────┼───────┤ H ├─■─────────X──»
                    │       ┌───┐ │        │P(π/2)  │P(π/4) └───┘           │  »
q_2: ──────■────────┼───────┤ H ├─┼────────■────────■───────────────────────X──»
     ┌───┐ │P(π/2)  │P(π/4) └───┘ │P(π/8)                                      »
q_3: ┤ H ├─■────────■─────────────■────────────────────────────────────────────»
     └───┘                                                                     »
«        
«q_0: ─X─
«      │ 
«q_1: ─┼─
«      │ 
«q_2: ─┼─
«      │ 
«q_3: ─X─
«

In [14]:
iqft = qft.inverse()
iqft.draw()

┌───┐                                                            »
q_0: ────X─┤ H ├─■──────────────■───────────────────■───────────────────────»
         │ └───┘ │P(-π/2) ┌───┐ │                   │                       »
q_1: ─X──┼───────■────────┤ H ├─┼─────────■─────────┼──────────────■────────»
      │  │                └───┘ │P(-π/4)  │P(-π/2)  │        ┌───┐ │        »
q_2: ─X──┼──────────────────────■─────────■─────────┼────────┤ H ├─┼────────»
         │                                          │P(-π/8) └───┘ │P(-π/4) »
q_3: ────X──────────────────────────────────────────■──────────────■────────»
                                                                            »
«                    
«q_0: ───────────────
«                    
«q_1: ───────────────
«                    
«q_2: ─■─────────────
«      │P(-π/2) ┌───┐
«q_3: ─■────────┤ H ├
«               └───┘

In [19]:
def qpe_for_s(n):
    qreg = QuantumRegister(2*n,'q')
    creg = ClassicalRegister(n,'c')
    qpe = QuantumCircuit(qreg, creg)
    qpe.h(qreg[0])
    qpe.cs(qreg[0],qreg[n])
    # qpe.cp(pi / 2, qreg[0], qreg[n])
    qpe.h(qreg[1])
    qpe.cz(qreg[1],qreg[n+1])
    for i in range(2,n):
        qpe.h(qreg[i])
    iqft_qpe = iqft_gater(qreg[0:n])
    qpe.append(iqft_qpe, qreg[0:n])
    qpe.measure(qreg[0:n],creg)
    return qpe

In [20]:
qpe = qpe_for_s(4)
simulator = Aer.get_backend('qasm_simulator')

job = execute(qpe, backend=simulator, shots=1024)
result = job.result()

counts = result.get_counts()

print("Measurement results:")
for state, count in counts.items():
    print(f"{state} (binary) -> {count} counts")

qpe.draw()

Measurement results:
0000 (binary) -> 1024 counts


┌───┐        ┌───────┐┌─┐         
q_0: ┤ H ├──■─────┤0      ├┤M├─────────
     ├───┤  │     │       │└╥┘┌─┐      
q_1: ┤ H ├──┼───■─┤1      ├─╫─┤M├──────
     ├───┤  │   │ │  IQFT │ ║ └╥┘┌─┐   
q_2: ┤ H ├──┼───┼─┤2      ├─╫──╫─┤M├───
     ├───┤  │   │ │       │ ║  ║ └╥┘┌─┐
q_3: ┤ H ├──┼───┼─┤3      ├─╫──╫──╫─┤M├
     └───┘┌─┴─┐ │ └───────┘ ║  ║  ║ └╥┘
q_4: ─────┤ S ├─┼───────────╫──╫──╫──╫─
          └───┘ │           ║  ║  ║  ║ 
q_5: ───────────■───────────╫──╫──╫──╫─
                            ║  ║  ║  ║ 
q_6: ───────────────────────╫──╫──╫──╫─
                            ║  ║  ║  ║ 
q_7: ───────────────────────╫──╫──╫──╫─
                            ║  ║  ║  ║ 
c: 4/═══════════════════════╩══╩══╩══╩═
                            0  1  2  3